# Basic Risk Simulation for Ergodicity Paper - Parallel Processing

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/research'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
from tqdm.notebook import tqdm  # progress bar
from itertools import product
from run_basic_simulation import run_basic_simulation

# Add these imports
import os
from joblib import Parallel, delayed

# Integrate joblib with tqdm for per-task progress
class TqdmJoblib(tqdm):
    def __enter__(self):
        from joblib import parallel
        self._original = parallel.BatchCompletionCallBack
        outer = self
        class TqdmBatchCompletionCallback(parallel.BatchCompletionCallBack):
            def __call__(self, *args, **kwargs):
                outer.update(n=self.batch_size)
                return super().__call__(*args, **kwargs)
        parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
        return super().__enter__()
    def __exit__(self, exc_type, exc, tb):
        from joblib import parallel
        parallel.BatchCompletionCallBack = self._original
        return super().__exit__(exc_type, exc, tb)

NUM_SIMULATIONS = 1_000
SIM_YEARS = 50
PRICING_SIMULATIONS = 500_000  # For premium estimation

initial_assets_list = [5_000_000, 10_000_000, 25_000_000]
atr_list = [0.8, 0.9, 1.0, 1.1, 1.2]
ebitabl_list = [0.1, 0.125, 0.15]
deductible_list = [0, 50_000, 100_000, 250_000, 500_000]
loss_ratio_list = [0.6, 0.7, 0.8]

total_combos = (
    len(initial_assets_list)
    * len(atr_list)
    * len(ebitabl_list)
    * len(deductible_list)
    * len(loss_ratio_list)
)

# Use all but one CPU core
_cpu = os.cpu_count() or 1
n_jobs = max(_cpu - 1, 1)

# Parallel execution with progress bar
with TqdmJoblib(total=total_combos, desc="Parameter sets", leave=True):
    Parallel(n_jobs=n_jobs, prefer="processes", batch_size=1)(
        delayed(run_basic_simulation)(
            index, ia, atr, ebitabl, ded, lr,
            NUM_SIMULATIONS, SIM_YEARS, PRICING_SIMULATIONS
        )
        for index, (ia, atr, ebitabl, ded, lr) in enumerate(product(
            initial_assets_list,
            atr_list,
            ebitabl_list,
            deductible_list,
            loss_ratio_list
        ))
    )


Parameter sets:   0%|          | 0/675 [00:00<?, ?it/s]